<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/Otro%20intento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install shap

In [2]:
import pandas as pd
import numpy as np
#import hyperopt
#from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

#from pandas.core.common import random_state
import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns # for correlation heatmap
#import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression as LR
from sklearn.neighbors import KNeighborsRegressor as KNN

SEED = 42

In [3]:
url='https://drive.google.com/file/d/1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_original = pd.read_csv(url)

In [ ]:
df_original.head()

In [ ]:
print (df_original.isnull().sum())

se remplazan los null por la media de la columna

In [6]:
edad_media = df_original['edad'].mean()
edad_media

45.02159532392986

In [7]:
def transfromar_DataFrame (df):
  df = df.copy()
  df = df.drop(df.columns[0],axis=1)
  #Remplazar nulls de la edad con la media de la columna
  df['edad'] = df['edad'].fillna(edad_media, inplace=False)

  #Transformar categóricas en numéricas
  df['peorcalificacionBCU'] = (df['peorcalificacionBCU'].
                              str.replace('2B','3').
                              str.replace('5','6').
                              str.replace('4','5').
                              str.replace('3','4').
                              str.replace('2A','2').
                              str.replace('1C','1').
                              str.replace('1A','0'))
  df['peorcalificacionBCU'] = pd.to_numeric(df['peorcalificacionBCU'], downcast='integer')
  return df

In [8]:
df = transfromar_DataFrame(df_original)


In [ ]:
df.head()

In [9]:
y = df.pop("ingreso")
X = df.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)


In [10]:
# La linea base es el promedio de los ingresos
baseline_preds = pd.DataFrame(y_train.copy())
media = y_train.mean()
baseline_preds['media'] = media
# Baseline errors, and display average baseline error
baseline_MAPE = MAPE(y_train,baseline_preds['media'])
print('MAPE linea base: ', round(baseline_MAPE *100, 2))

MAPE linea base:  58.03


In [23]:
# Instantiate model with 1000 decision trees
rf0 = RandomForestRegressor(n_estimators = 500, random_state = SEED)
#rf1 = RandomForestRegressor(n_estimators = 500, criterion="absolute_error", max_features=3, bootstrap=True, oob_score=True, random_state = SEED)
#rf2 = RandomForestRegressor(n_estimators = 500, max_features=5, bootstrap=True, oob_score=False, random_state = SEED)


# Train the model on training data
rf0.fit(X_train,y_train)
#rf1.fit(X_train,y_train)
#rf2.fit(X_train,y_train)

RandomForestRegressor(n_estimators=500, random_state=42)

In [ ]:
help(RandomForestRegressor)

In [24]:
# Use the forest's predict method on the test data
preds_rf0 = rf0.predict(X_test)
# Calculate the absolute percentage errors
MAPE_rf0 = MAPE(y_test, preds_rf0)
# Print out the mean absolute percentageerror (mape)
print('MAPE_rf0:', round(100*MAPE_rf0, 2))

MAPE_rf0: 12.93


**Optimización del modelo**

In [25]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 650, 1100, 1550, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 43, 76, 110, None], 'min_samples_split': [10], 'min_samples_leaf': [4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_opt = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_opt, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [1]:
rf_random.best_params_

NameError: ignored

In [ ]:
# Evaluar el modelo optimizado vs modelo base

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print(('MAPE:', round(100*mape, 2))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return mape

base_accuracy = evaluate(rf0, X_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
preds_opt = best_random.predict(X_test)
# Calculate the absolute percentage errors
MAPE_opt = MAPE(y_test, preds_opt)
# Print out the mean absolute percentageerror (mape)
print('MAPE_opt:', round(100*MAPE_opt, 2))

**Cargamos el data final y predecimos**

In [ ]:
url='https://drive.google.com/file/d/1sMkorQLOmf2cNGRwXkQLKeciGSUpL_ez/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_final = pd.read_csv(url)

df_final.head()

In [ ]:
df_final = transfromar_DataFrame(df_final)

In [ ]:
preds_datathon2 = best_random.predict(df_final)

In [ ]:
pd.DataFrame(preds_datathon2).to_csv('predicciones2.csv') 